In [ ]:
##Processing spectral data cube files
#This is intended to process a collection of .cube and .hdr files generated from a Surface Optics Camera on motorized rail
#A set of file names should be created as a text file
#A step in this process is creating a calibration file in ENVI. That step is not detailed here.
#Matthew Clarke, National Museum of Asian Art, Smithsonian Institution, 2025-12

In [ ]:
#Import libraries
import os
%matplotlib qt 
import numpy as np
import matplotlib.pyplot as plt
from spectral import *
import csv
import re

In [ ]:
#Get the directory, field file
os.chdir('D:\\Processing-XRF\\Datafolder') #set the location of the files; sometimes this needs \\
flatfile='flatfield_cubefile' #no extension 

In [ ]:
#Open the flatfield file, get mean, create new image cube that matches the calibration standards image
sflat=envi.open(flatfile+'.hdr',flatfile+'.cube')
sarrflat = sflat.load()
#Get the mean of the movement axis
sarrflatmean=np.mean(sarrflat, axis=0) #get the aveage of the movement axis
totalrows=256 #this should match calibration standards image size
sarrflatmeanbig = np.repeat(sarrflatmean[:, np.newaxis, :], totalrows, axis=1)
sarrflatmeanbig = np.swapaxes(sarrflatmeanbig, 0,1) #need to rearrange axis to match original data order
metaout = sflat.metadata.copy()
metaout['bands']=str(totalrows)
flatfileout=flatfile[0:-6] #remove existing extension
envi.save_image(flatfileout+'_mean.hdr', sarrflatmeanbig, dtype=np.float32, interleave='bil', metadata=metaout, ext='.dat')

In [1]:
##Now complete calibration processing in ENVI
#This is using spectral math S1/S2, where S1 is the calibration image, and S2 is the flatfiled
#Next perform empirical line calibration based on reflectance standards.
#Save the .cff file for use in batch processing of all sample cubes.
#Create a txt file of the sample cubes for batch processing. This is the .hdr files list only.

In [ ]:
#Open the cff file and sample file list.
calfile='cff_RefStds_VNIR_20241216_160ms_f4_Effilux-100.cff' #with cff ext; need to create in ENVI 

filelistfile='batchcontents.txt' #list of .hdr files

In [ ]:
#Make a list of files as an array that will be processed
with open(filelistfile, newline='') as f:
    reader = csv.reader(f)
    filelist = list(reader)
filelist[0]

In [ ]:
#Open the calibration and flat field files
calarr=np.genfromtxt(calfile, skip_header=5, delimiter="")
sflat=envi.open(flatfile+'.hdr',flatfile+'.cube')

In [ ]:
##Load Sample cubes and flat field and calibrate and crop edge pixels, and crop spectra, save new cubes.
#This data has cubes from the VNIR chip = 1024x1024, and 256 bands.
#The spectral response past 1000 nm was very low signal as the NIR LEDs do not extend this far. This is cropped from final cubes.
#cropping the ends of the spatial part of the cube can eliminate low signal areas and lines where the camera movement is not yet consistent. 
rowstart = 4 #0 index
rowend = 1020 #this is +1 of end
samplestart = 4
sampleend = 1020
bandstart = 0
bandend = 236
for fileiter in range(0, len(filelist)): #range needs to go to end
    datafile=str(filelist[fileiter])
    datafile=datafile[2:-6] #remove existing extension
    sdata=envi.open(datafile+'.hdr',datafile+'.cube')
    totalrows = sdata.metadata['lines'] #this should match the sample total rows
    sarrflatmeanbig = np.repeat(sarrflatmean[:, np.newaxis, :], totalrows, axis=1)
    sarrflatmeanbig = np.swapaxes(sarrflatmeanbig, 0,1) #need to rearrange axis to match original data order
    sarrdata = sdata.load()
    sarrdataproc=np.ndarray(shape=sarrdata.shape, dtype=float)
    #calibrate file
    for b in range(0, sarrdata.shape[2]): #range needs to go to end
        sarrdataproc[:,:,b]=((np.squeeze(sarrdata[:,:,b])/np.squeeze(sarrflatmeanbig[:,:,b]))-calarr[b,2])/calarr[b,1] #correct formula
    #crop file
    subproc=sarrdataproc[rowstart:rowend, samplestart:sampleend, bandstart:bandend]
    cropmeta = sdata.metadata.copy()
    cropmeta['wavelength']=sdata.metadata['wavelength'][bandstart:bandend]
    cropmeta['bands']=str(bandend-bandstart)
    cropmeta['samples']=str(sampleend-samplestart)
    cropmeta['lines']=str(rowend-rowstart)
    #Save the file with cropping
    calcropout='crcff_'+datafile+'.hdr'
    envi.save_image(calcropout, subproc, dtype=np.float32, interleave='bil', metadata=cropmeta, ext='.dat')
    print(fileiter)